In [7]:
pip install nnunetv2 pydicom nibabel torch torchvision torchaudio

  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
   -------------------------------

In [11]:
!pip install nnunetv2 torch torchvision torchaudio pydicom nibabel matplotlib


  Using cached argparse-1.4.0-py2.py3-none-any.whl.metadata (2.8 kB)
Using cached argparse-1.4.0-py2.py3-none-any.whl (23 kB)


In [15]:
import os
import nibabel as nib
import numpy as np
import matplotlib.pyplot as plt
import pydicom
from pydicom.filereader import dcmread
import subprocess

# ✅ Update this to your folder path
dicom_folder = r"C:\Users\anude\OneDrive\Desktop\project\ProstateX-0000\07-07-2011-NA-MR prostaat kanker detectie WDSmc MCAPRODETW-05711\3.000000-t2tsesag-87368"
nifti_file = "input_image.nii.gz"
output_dir = r"C:\Users\anude\OneDrive\Desktop\project"


In [17]:
def dicom_series_to_nifti(dicom_folder, output_path):
    slices = []
    for file in sorted(os.listdir(dicom_folder)):
        path = os.path.join(dicom_folder, file)
        try:
            ds = dcmread(path)
            if hasattr(ds, 'pixel_array'):
                slices.append(ds)
        except Exception as e:
            print(f"Skipped {file}: {e}")
    if not slices:
        raise RuntimeError("No valid DICOM slices found.")
    slices.sort(key=lambda x: float(x.ImagePositionPatient[2]))
    img_3d = np.stack([s.pixel_array for s in slices])
    affine = np.eye(4)
    nifti_img = nib.Nifti1Image(img_3d, affine)
    nib.save(nifti_img, output_path)
    print(f"[INFO] NIfTI saved to: {output_path}")

# Run conversion
dicom_series_to_nifti(dicom_folder, nifti_file)


[INFO] NIfTI saved to: input_image.nii.gz


In [33]:
import os

# Replace this path with your actual nnUNet results directory
nnUNet_base = r"C:\Users\anude\OneDrive\Desktop"  
# or any path where models will live

# Set environment variables for nnU-Net
os.environ["nnUNet_raw"] = os.path.join(nnUNet_base, "nnUNet_raw")
os.environ["nnUNet_preprocessed"] = os.path.join(nnUNet_base, "nnUNet_preprocessed")
os.environ["nnUNet_results"] = os.path.join(nnUNet_base, "nnUNet_results")

# Create folders if they don't exist
os.makedirs(os.environ["nnUNet_raw"], exist_ok=True)
os.makedirs(os.environ["nnUNet_preprocessed"], exist_ok=True)
os.makedirs(os.environ["nnUNet_results"], exist_ok=True)


In [35]:
def run_nnunet_prediction(input_path, output_dir):
    os.makedirs(output_dir, exist_ok=True)
    cmd = [
        "nnUNetv2_predict",
        "-i", input_path,
        "-o", output_dir,
        "-d", "005",
        "-c", "3d_fullres",
        "-f", "0"
    ]
    print("[INFO] Running nnU-Net prediction...")
    try:
        result = subprocess.run(cmd, check=True, capture_output=True, text=True)
        print(result.stdout)
    except subprocess.CalledProcessError as e:
        print("[ERROR] nnUNetv2_predict failed:")
        print(e.stderr)
run_nnunet_prediction(".", output_dir)

[INFO] Running nnU-Net prediction...
[ERROR] nnUNetv2_predict failed:
Traceback (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "C:\Users\anude\anaconda3\Scripts\nnUNetv2_predict.exe\__main__.py", line 7, in <module>
  File "C:\Users\anude\anaconda3\Lib\site-packages\nnunetv2\inference\predict_from_raw_data.py", line 948, in predict_entry_point
    model_folder = get_output_folder(args.d, args.tr, args.p, args.c)
                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\anude\anaconda3\Lib\site-packages\nnunetv2\utilities\file_path_utilities.py", line 22, in get_output_folder
    tmp = join(nnUNet_results, maybe_convert_to_dataset_name(dataset_name_or_id),
                               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\anude\anaconda3\Lib\site-packages\nnunetv2\utilities\dataset_name_id_conversion.py", line 74, in maybe_convert_to_datas

In [9]:
python dicom_to_nifti.py --dicom_folder dicom_input --output_file input_image.nii.gz


SyntaxError: invalid syntax (3337603165.py, line 1)

In [3]:
import os
import argparse
import nibabel as nib
import numpy as np
import pydicom
import subprocess
import matplotlib.pyplot as plt
from pydicom.filereader import dcmread

def dicom_series_to_nifti(dicom_folder, output_path):
    slices = []
    for file in sorted(os.listdir(dicom_folder)):
        path = os.path.join(dicom_folder, file)
        try:
            ds = dcmread(path)
            if hasattr(ds, 'pixel_array'):
                slices.append(ds)
        except Exception as e:
            print(f"Skipped {file}: {e}")
    if not slices:
        raise RuntimeError("No valid DICOM slices found.")
    slices.sort(key=lambda x: float(x.ImagePositionPatient[2]))
    img_3d = np.stack([s.pixel_array for s in slices])
    affine = np.eye(4)
    nifti_img = nib.Nifti1Image(img_3d, affine)
    nib.save(nifti_img, output_path)
    print(f"Saved NIfTI to {output_path}")

def run_nnunet_prediction(input_path, output_dir):
    cmd = [
        "nnUNetv2_predict",
        "-i", input_path,
        "-o", output_dir,
        "-d", "005",  # Task005_Prostate
        "-c", "3d_fullres",
        "-f", "0"
    ]
    print("Running nnU-Net prediction...")
    subprocess.run(cmd, check=True)
    print("Prediction completed!")

def visualise_segmentation(seg_file):
    print("Loading prediction for visualisation...")
    seg = nib.load(seg_file)
    data = seg.get_fdata()
    middle_slice = data.shape[2] // 2
    plt.imshow(data[:, :, middle_slice])
    plt.title("Predicted Segmentation (Middle Slice)")
    plt.axis('off')
    plt.show()

if __name__ == "__main__":
    parser = argparse.ArgumentParser(description="Predict Prostate MRI DICOM with nnU-Net")
    parser.add_argument("--dicom_folder", type=str, required=True, help="Folder with DICOM slices")
    parser.add_argument("--output_dir", type=str, default="output", help="Where to save the prediction")
    parser.add_argument("--visualise", action="store_true", help="Visualise the prediction")
    args = parser.parse_args()

    nifti_file = "input_image.nii.gz"

    dicom_series_to_nifti(args.dicom_folder, nifti_file)
    os.makedirs(args.output_dir, exist_ok=True)
    run_nnunet_prediction(".", args.output_dir)

    if args.visualise:
        seg_output = os.path.join(args.output_dir, "input_image.nii.gz")
        visualise_segmentation(seg_output)


ModuleNotFoundError: No module named 'nibabel'